# EXAMPLE 

For practical implementation:
https://scikit-learn.org/stable/auto_examples/linear_model/plot_polynomial_interpolation.html

For theoretical knowledge of spline:
https://timodenk.com/blog/cubic-spline-interpolation/


In [7]:
# Author: Mathieu Blondel
#         Jake Vanderplas
#         Christian Lorentzen
#         Malte Londschien
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
from sklearn.pipeline import make_pipeline


# Trial on Day-ahead data

So what is going to happen is as follows:

0. Try to plot day-ahead price
1. Try fit a periodic spline on day-ahead data
2. Pick 5 spots on the day-ahead price and add noise to it. Then fit a periodic spline function to it afterwards

Open the real.csv file and the day-ahead

### Loading data

In [8]:
data = pd.read_csv('real.csv') # Change path
data = data.fillna(0)
spot = data["Spot"].to_numpy()
spot_reshape = np.reshape(spot, (24, 90+365),order ='F') # F to ensure the correct reshaping.
spot_reshaped_forecasted = spot_reshape[:,365:]
mean_spot_all_days = np.mean(spot_reshaped_forecasted,axis=1)
std_spot_all_days = np.std(spot_reshaped_forecasted,axis = 1)


print(np.shape(spot_reshape[:,365:]))


(24, 90)


## Deciding on the training data

In [ ]:
x_train = np.array([0,3,8,13,18,23])
y_train = mean_spot_all_days[x_train]
plt.plot(mean_spot_all_days)
plt.scatter(x_train, y_train,c='black', label="training points")
plt.ylabel("Price [EUR]")

# Set the plot title
title = "Mean Spot price "
plt.title(title, fontsize=18)

## Approximate with spline function

In [ ]:
x_train = np.array([0,3,8,13,18,23]) # Data which is traded upon
X_train = x_train[:, np.newaxis] # necessary for the scikit method to work
y_train = spot_reshaped_forecasted[x_train,1]

x_plot = np.arange(24) # For plotting purposes
X_plot = x_plot[:, np.newaxis]


Building the model:

In [ ]:

model = make_pipeline(SplineTransformer(n_knots=6, degree=3), Ridge(alpha=1e-3))
model.fit(X_train, y_train)

y_plot = model.predict(X_plot)


In [ ]:
plt.plot(spot_reshaped_forecasted[:,1],label = "Day-ahead price")
plt.plot(y_plot,label = "Forecast - No noise")
plt.scatter(x_train, y_train,c='black', label="training points")
plt.legend(loc="lower center")

# ADDING NOISE TO CREATE THE FORECASTS

First we assess the standard deviation of the time instances of our data, because then we know how much noise we should add.


In [ ]:
print(np.reshape(std_spot_all_days[x_train],(6,1)))

However the standard deviation is way too much, so let us take a percentage of it. For

- D-2, 30 %
- DA, 20 %
- D-1, 10 %

In [ ]:
std_train = std_spot_all_days[x_train]*0.1
noise = np.random.normal(0, std_train, std_train.shape)
print(np.shape(noise))
print(noise)

In [9]:
def Forecast_of_spot(Spot_data):

    '''
    
    Spot_data (dataframe). NEED TO BE DIVISIBLE OF 24

    '''

    # Load the data
    spot = Spot_data.to_numpy() # Transform to numpy
    spot_reshape = np.reshape(spot, (24, int(len(spot)/24)),order ='F') # F to ensure the correct reshaping
    mean_spot_all_days = np.mean(spot_reshape,axis=1)
    std_spot_all_days  = np.std(spot_reshape  ,axis = 1)

    # Set the training data settings
    x_train = np.array([0,3,8,13,18,23]) # Data which is traded upon
    X_train = x_train[:, np.newaxis] # necessary for the scikit method to work

    # Construct the noise
    std_train_D_2 = std_spot_all_days[x_train]*0.2
    std_train_D_1 = std_spot_all_days[x_train]*0.1
    noise_D_2 = np.random.normal(0, std_train_D_2, std_train_D_2.shape)
    noise_D_1 = np.random.normal(0, std_train_D_1, std_train_D_1.shape)

    # Set up the calculated parts
    True_Spot= spot_reshape 
    Forecast_no_noise = np.zeros((24,90))
    Forecast_D_2 = np.zeros((24,90))
    Forecast_D_1 = np.zeros((24,90))


 
   # Set up settings for the Spline interpolation
    x_day = np.arange(24) # For plotting purposes
    X_day = x_day[:, np.newaxis]

    model = make_pipeline(SplineTransformer(n_knots=6, degree=3), Ridge(alpha=1e-3))

    days = len(True_Spot[0,:])
    for d in range(0,days):

        # Forecast with noise, at D-2. Two days prior
        y_train_D_2 = True_Spot[x_train,d] + noise_D_2
        model.fit(X_train, y_train_D_2) 
        y_day = model.predict(X_day) 
        Forecast_D_2[:,d] = np.reshape(y_day, (24,) ) # Save the forecast

        # Forecast with noise, at D-1. One days prior
        y_train_D_1 = True_Spot[x_train,d] + noise_D_1
        model.fit(X_train, y_train_D_1) 
        y_day = model.predict(X_day) 
        Forecast_D_1[:,d] = np.reshape(y_day, (24,) ) # Save the forecast

        # Forecast without noise
        model.fit(X_train, spot_reshape[x_train,d]) 
        y_day = model.predict(X_day)
        Forecast_no_noise[:,d] = np.reshape(y_day, (24,) ) # Save the forecast


    
    return Forecast_D_2, Forecast_D_1, Forecast_no_noise, True_Spot

In [ ]:
Forecast_D_2, Forecast_D_1, Forecast_no_noise, True_Spot = Forecast_of_spot(data["Spot"].loc[(24*365):])

Visualize the forecasts

In [ ]:

# Set the font size of the plot
plt.rcParams.update({'font.size': 10})

# Set the figure size and dpi
fig = plt.figure(figsize=(8, 6), dpi=100)

# Plot the data
d = 10
plt.plot(True_Spot[:,d],  label="Day-ahead price")
plt.plot(Forecast_D_2[:,d],  label="Forecast 2 days prior")
plt.plot(Forecast_D_1[:,d],  label="Forecast 1 day prior")
plt.scatter(x_train, Forecast_D_2[x_train,d], c='black', s=10)
plt.scatter(x_train, Forecast_D_1[x_train,d], c='black', s=10)

# Add a legend outside of the plots
fig.legend(loc='lower center', bbox_to_anchor=(0.5, -0.001), ncol=3)

plt.ylabel("Price [EUR]")

# Set the plot title
title = "Spot price and forecast at day {}".format(d)
plt.title(title, fontsize=18)

# Show the plot

plt.savefig('Spot_Forecast.png')
plt.show()


## Percentage difference:

In [ ]:
import numpy as np

avg_percentage = np.mean(np.abs(True_Spot - Forecast_D_2) / True_Spot, axis=0)
print(np.mean(avg_percentage))
avg_percentage = np.mean(np.abs(True_Spot - Forecast_D_1) / True_Spot, axis=0)
print(np.mean(avg_percentage))
for d, avg in enumerate(avg_percentage):
    print("Day", d, avg)


In [ ]:

# Set the font size of the plot
plt.rcParams.update({'font.size': 10})

# Set the figure size and dpi
fig = plt.figure(figsize=(8, 6), dpi=100)

# Plot the data
d = 0
plt.plot(True_Spot[:,d],  label="Day-ahead price")
plt.plot(Forecast_D_2[:,d],  label="Forecast 2 days prior")
plt.plot(Forecast_D_1[:,d],  label="Forecast 1 day prior")
plt.scatter(x_train, Forecast_D_2[x_train,d], c='black', s=10)
plt.scatter(x_train, Forecast_D_1[x_train,d], c='black', s=10)

# Add a legend outside of the plots
fig.legend(loc='lower center', bbox_to_anchor=(0.5, -0.001), ncol=3)

plt.ylabel("Price [EUR]")

# Set the plot title
title = "Spot price and forecast at day {}".format(d)
plt.title(title, fontsize=18)

# Show the plot

plt.show()


Save it all into one dataframe:

In [ ]:
True_Spot_reshaped = pd.DataFrame(np.reshape(True_Spot, (24*90,), order = 'F'),columns=["Spot"])

Forecast_D_2_reshaped = pd.DataFrame(np.reshape(Forecast_D_2, (24*90,), order = 'F'),columns=["Spot_D_2"])
Forecast_D_1_reshaped = pd.DataFrame(np.reshape(Forecast_D_1, (24*90,), order = 'F'),columns=["Spot_D_1"])







# Set up FCR-D forecast. Mean for the past 5 days.
- At D-2 it is past 5 days, meaning 7 days from what to predict.
- At D-1 it is the past 5 days, meaning 6 days from what to predict

In [10]:
data = pd.read_csv('real.csv') # Change path
data = data.fillna(0) 
data_cop = data.copy()
data = data.drop(columns=["Hour"])
setup_forecast = data_cop.drop(columns=["Hour"])
True_data = data.loc[8760:].reset_index(drop=True)
True_data.columns
Spot_data = True_data['Spot']


In [24]:

Total_length_test = len(data["FD1_down"]) - 8760
# Create an empty dataframe with the same columns as `data`
forecasted_data_D_2 = pd.DataFrame(columns=setup_forecast.columns, index=range(Total_length_test))
forecasted_data_D_1 = pd.DataFrame(columns=setup_forecast.columns, index=range(Total_length_test))
True_data = pd.DataFrame(columns=setup_forecast.columns, index=range(Total_length_test))

print(forecasted_data_D_2.columns)

Index(['Spot', 'FD1_down', 'FD2_down', 'FD1_up', 'FD2_up', 'FD1_up_percentage',
       'FD2_up_percentage', 'FD1_down_percentage', 'FD2_down_percentage',
       'FD_act_up', 'FD_act_down'],
      dtype='object')


In [30]:

D_2_list = 24*[3,4,5,6,7]
D_1_list = 24*[2,3,4,5,6]
for col in data.columns:
    if col == "Spot":
        # Run spline procedure
        # Use True_data
        print("Spot started")
        Forecast_Spot_D_2, Forecast_Spot_D_1, Forecast_Spot_no_noise, True_Spot = Forecast_of_spot(Spot_data)
        Forecast_D_2_reshaped = pd.DataFrame(np.reshape(Forecast_Spot_D_2, (24*90,), order = 'F'),columns=["Spot"])
        forecasted_data_D_2[col] = Forecast_D_2_reshaped
        Forecast_D_1_reshaped = pd.DataFrame(np.reshape(Forecast_Spot_D_1, (24*90,), order = 'F'),columns=["Spot"] )
        forecasted_data_D_1[col] = Forecast_D_1_reshaped
        True_Spot_reshaped = pd.DataFrame(np.reshape(True_Spot, (24*90,), order = 'F'),columns=["Spot"] )
        True_data = True_Spot_reshaped
        print("Spot ended")
    else: 
        print(col, " started")
        # Run mean procedure
        for h in range(0,Total_length_test):
            True_data.loc[h, col] = data.loc[h+8760,col]
            # As the volumes need to add up to 100% then is it only the D-1 which has been forecasted as t
            if 'percentage' in col:
                if "FD2" in col:
                    if "up" in col:
                        forecasted_data_D_2.loc[h, col] = 1 - forecasted_data_D_2.loc[h, 'FD1_up_percentage']
                        forecasted_data_D_1.loc[h, col] = 1 - forecasted_data_D_1.loc[h, 'FD1_up_percentage']
                    else:
                        forecasted_data_D_2.loc[h, col] = 1 - forecasted_data_D_2.loc[h, 'FD1_down_percentage']
                        forecasted_data_D_1.loc[h, col] = 1 - forecasted_data_D_1.loc[h, 'FD1_down_percentage']
                else:
                    forecasted_data_D_2.loc[h, col] = np.mean([data.loc[h+8760-hd, col] for hd in D_2_list])
                    forecasted_data_D_1.loc[h, col] = np.mean([data.loc[h+8760-hd, col] for hd in D_1_list])

            else:
                forecasted_data_D_2.loc[h, col] = np.mean([data.loc[h+8760-hd, col] for hd in D_2_list])
                forecasted_data_D_1.loc[h, col] = np.mean([data.loc[h+8760-hd, col] for hd in D_1_list])
                
        print(col, " ended")

                
        

Spot started
Spot ended


# Quantify the predictability of the forecasts

Create a histogram of the precision
Remove all true values which are 0.

In [31]:
Percentage_data = pd.DataFrame(columns=['Spot', 'FD1_down', 'FD2_down', 'FD1_up', 'FD2_up'], index=range(Total_length_test))

for col in ['Spot', 'FD1_down', 'FD2_down', 'FD1_up', 'FD2_up']:
    print(col)
    # why inf.....
    indexes = np.where(true != 0)[0] # Index to NOT remove

    pred = forecasted_data_D_2[col].loc[indexes].values
    print(pred)
    true = True_data[col].loc[indexes].values


    avg_percentage = np.mean(np.abs(true - pred) / true)
    Percentage_data[col] = np.abs(true - pred) / true
    print(avg_percentage)



# Assuming avg_percentage is your array
#masked_avg_percentage = avg_percentage[np.logical_and(np.isfinite(avg_percentage), ~np.isnan(avg_percentage))]
#mean = np.mean(masked_avg_percentage)

#print(mean)



Spot
[-12.07487191  -7.86830604  -1.59208672 ...  66.47764887  40.68190022
  21.14811114]
0.5321396943595096
FD1_down
[323.5532579999999 336.005888 274.50441799999993 ... 11.633944000000001
 10.740552000000001 9.770456000000001]
inf
FD2_down
[53.08922 55.571813999999996 58.03772000000001 ... 18.223052
 18.242701999999998 18.243099999999995]


C:\Users\Daniel  BV\AppData\Local\Temp\ipykernel_25752\1798536061.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  avg_percentage = np.mean(np.abs(true - pred) / true)
C:\Users\Daniel  BV\AppData\Local\Temp\ipykernel_25752\1798536061.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  Percentage_data[col] = np.abs(true - pred) / true


ValueError: Length of values (2158) does not match length of index (2160)

In [ ]:
True_FCRD_vol = data[["FD1_down_percentage","FD2_down_percentage","FD1_up_percentage","FD2_up_percentage"]].loc[8760:].reset_index(drop=True)


fig, axs = plt.subplots(2, 2, figsize=(10, 8), dpi=100)
d = 10
for i, col in enumerate(True_FCRD_vol.columns):
    ax = axs[i // 2, i % 2]
    ax.plot(True_FCRD_vol[col].iloc[int(0+float(d)*24):int(24+float(d)*24)]*100, label="True FCR-D volumes")
    ax.plot(forecasted_data_D_2[col].iloc[int(0+float(d)*24):int(24+float(d)*24)]*100, label="Forecast 2 days prior")
    ax.plot(forecasted_data_D_1[col].iloc[int(0+float(d)*24):int(24+float(d)*24)]*100, label="Forecast 1 day prior")
    ax.set_ylabel(col)

axs[0,0].set_ylabel('D-1 up Volume [%]') 
axs[0,1].set_ylabel('D-2 up Volume [%]') 
axs[1,0].set_ylabel('D-1 dn Volume [%]') 
axs[1,1].set_ylabel('D-2 dn Volume [%]') 


# Add a legend outside of the plots
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.001), ncol=3)

# Set the plot title
title = "FCR-D Volumes and forecast at day {}".format(d)
fig.suptitle(title, fontsize=18)

# Show the plot
plt.show()
fig.savefig('FCR-D_Forcasts_volumes.png')

In [ ]:
import seaborn as sns

# Sample data
data = df["Spot"]

# Set up seaborn style
sns.set(style="whitegrid")

# Create histogram
sns.histplot(data, kde=True, color='skyblue')

# Add labels and title
plt.xlabel('Frequency [Hz]')
plt.ylabel('Occurences')

# Show the plot
plt.show()


In [ ]:
True_FCRD = data[["FD1_down","FD2_down","FD1_up","FD2_up"]].loc[8760:].reset_index(drop=True)

fig, axs = plt.subplots(2, 2, figsize=(10, 8), dpi=100)
d = 10
for i, col in enumerate(True_FCRD.columns):
    ax = axs[i // 2, i % 2]
    ax.plot(True_FCRD[col].iloc[int(0+float(d)*24):int(24+float(d)*24)], label="True FCR-D prices")
    ax.plot(forecasted_data_D_2[col].iloc[int(0+float(d)*24):int(24+float(d)*24)], label="Forecast 2 days prior")
    ax.plot(forecasted_data_D_1[col].iloc[int(0+float(d)*24):int(24+float(d)*24)], label="Forecast 1 day prior")
    ax.set_ylabel(col)

axs[0,0].set_ylabel('D-1 up price [EUR/MW]') 
axs[0,1].set_ylabel('D-2 up price [EUR/MW]') 
axs[1,0].set_ylabel('D-1 dn price [EUR/MW]') 
axs[1,1].set_ylabel('D-2 dn price [EUR/MW]') 


# Add a legend outside of the plots
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.001), ncol=3)

# Set the plot title
title = "FCR-D prices and forecast at day {}".format(d)
fig.suptitle(title, fontsize=18)

# Show the plot
plt.show()
fig.savefig('FCR-D_Forcasts_prices.png')

In [ ]:
True_act = data[["FD_act_up","FD_act_down"]].loc[8760:].reset_index(drop=True)

fig, axs = plt.subplots(2, 1, figsize=(10, 8), dpi=100)
d = 10
for i, col in enumerate(True_act.columns):
    ax = axs[i]
    ax.plot(True_act[col].iloc[int(0+float(d)*24):int(24+float(d)*24)], label="True activation")
    ax.plot(forecasted_data_D_2[col].iloc[int(0+float(d)*24):int(24+float(d)*24)], label="Forecast 2 days prior")
    ax.plot(forecasted_data_D_1[col].iloc[int(0+float(d)*24):int(24+float(d)*24)], label="Forecast 1 day prior")
    ax.set_ylabel(col)

axs[0].set_ylabel('D-1 up price [EUR/MW]')  
axs[1].set_ylabel('D-1 dn price [EUR/MW]')


# Add a legend outside of the plots
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.001), ncol=3)

# Set the plot title
title = "Activation and forecast at day {}".format(d)
fig.suptitle(title, fontsize=18)

# Show the plot
plt.show()
fig.savefig('Activation_Forecasts.png')

# Create a csv file for forecasted values of D-2

In [ ]:

forecasted_data_D_2 = forecasted_data_D_2
Forecast_Spot_D_2_reshaped = pd.DataFrame(np.reshape(Forecast_Spot_D_2, (24*90,), order = 'F'),columns=["Spot"])
print(forecasted_data_D_2.columns)
print(Forecast_Spot_D_2_reshaped.columns)
forecast_D_2_comb = pd.concat([Forecast_Spot_D_2_reshaped, forecasted_data_D_2], axis=1)
print(forecast_D_2_comb.shape)
print(forecast_D_2_comb.columns)

#Create csv
forecast_D_2_comb.to_csv("forecast.csv",index=False)